# Langchain X PaLM

This notebook contains the code for the LangChain + PaLM tutorial available on https://yactouat.com/langchain-palm-getting-started.

## Setting things up

In [104]:
from dotenv import load_dotenv
from google.cloud import aiplatform
import langchain
from langchain.chat_models import ChatVertexAI
from langchain.embeddings import VertexAIEmbeddings
from langchain.llms import VertexAI
from langchain.schema import AIMessage, HumanMessage, SystemMessage
import os
import time
# supporting type hints
from typing import List
import vertexai

load_dotenv()

True

In [105]:
# `GCP_LOCATION` is for instance `europe-west1`
vertexai.init(project=os.getenv('GCP_PROJECT_ID'), location=os.getenv('GCP_LOCATION')) 

print(f"Vertex AI SDK version: {aiplatform.__version__}")

print(f"LangChain version: {langchain.__version__}")

Vertex AI SDK version: 1.37.0
LangChain version: 0.0.346


In [106]:
# defining a function to rate limit API calls
def rate_limit(max_per_minute):
    period = 60 / max_per_minute
    print("Waiting")
    while True:
        before = time.time()
        # the `yield` keyword here controls back to the caller;
        # e.g. it allows the caller to perform the action that is rate limited
        yield
        # at this points, control returns to this function
        after = time.time()
        elapsed = after - before
        sleep_time = max(0, period - elapsed)
        if sleep_time > 0:
            print(".", end="")
            time.sleep(sleep_time)

In [107]:
class CustomVertexAIEmbeddings():
    def __init__(self, requests_per_minute: int, num_instances_per_batch: int):
        self.requests_per_minute = requests_per_minute
        self.num_instances_per_batch = num_instances_per_batch

    # takes a list of strings and returns a list of embeddings
    def embed_documents(self, texts: List[str]):
        limiter = rate_limit(self.requests_per_minute)
        results = []
        remaining = list(texts)

        while remaining:
            # working in batches because the API accepts maximum 5
            # documents per request to get embeddings
            batch, remaining = (
                # batch contains the first `num_instances_per_batch` documents
                remaining[: self.num_instances_per_batch],
                # docs contains the remaining documents
                remaining[self.num_instances_per_batch :],
            )
            self.vertex_ai_embeddings = VertexAIEmbeddings()
            chunk = self.vertex_ai_embeddings.client.get_embeddings(batch)
            results.extend(chunk)
            next(limiter)

        return [r.values for r in results]

In [108]:
# create a foundational LLM instance
# LLM model
llm = VertexAI(
    model_name="text-bison@002",
    max_output_tokens=2048,
    # setting the temperature that low will make the mode more deterministic
    temperature=0.5,
    # (nucleus sampling): the model will only consider a cumulative probability threshold above 0.8 to consider words to use
    top_p=0.8,
    # the model will only consider the top k most likely words when generating text
    top_k=40,
    verbose=True,
)


In [109]:
# set embeddings
# `QPM` stands for "queries per minute"
EMBEDDING_QPM = 100
EMBEDDING_NUM_BATCH = 5
embeddings = CustomVertexAIEmbeddings(
    requests_per_minute=EMBEDDING_QPM,
    num_instances_per_batch=EMBEDDING_NUM_BATCH,
)

## Basic interactions with an LLM

Text is the natural way of interacting with LLMs, so let's just do that:

In [110]:
interaction = "what is the future of AI multi modality?"
response = llm(interaction)

response

" The future of AI multimodality holds immense potential for advancing various fields and transforming our interactions with technology. Here are some key areas where AI multimodality is expected to make significant impacts:\n\n**Enhanced Human-Computer Interaction:** AI multimodality will enable more natural and intuitive interactions between humans and machines. By combining different modalities such as speech, gestures, facial expressions, and touch, AI systems can better understand user intent and provide more personalized and responsive experiences. This will revolutionize user interfaces, making them more user-friendly and accessible.\n\n**Improved Accessibility:** AI multimodality can significantly enhance accessibility for individuals with disabilities. By providing alternative input and output methods, such as voice commands, gesture recognition, and haptic feedback, AI systems can make technology more inclusive and accessible to a wider range of users.\n\n**Enhanced Healthcar

## Chat interactions

In [111]:
chat = ChatVertexAI()

initial_human_message = HumanMessage(
    content="What is the future of AI multi modality?",
)

chat_interaction = [
    SystemMessage(
        content="You are an AI assistant that is specialized in software enginering and in machine learning. Your answers are two short sentences long maximum.",
    ),
    initial_human_message
]

response = chat(chat_interaction)

print(response.content)

 Multimodal AI systems will become increasingly sophisticated, enabling them to process and understand a wider range of data types, including text, images, audio, and video. This will lead to a new generation of AI applications that can interact with the world in more natural and intuitive ways.


In [112]:
response_with_history = chat([
    SystemMessage(
        content="You are an AI assistant that is specialized in software enginering and in machine learning. Your answers are two short sentences long maximum.",
    ),
    initial_human_message,
    AIMessage(
        content=response.content,
    ),
    HumanMessage(content="what wider range of data types?")
])

print(response_with_history.content)

 Multimodal AI systems will be able to process and understand a wider range of data types, including text, images, audio, video, and even sensory data such as touch and smell. This will allow them to interact with the world in a more natural and intuitive way.
